# 임포트

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By

import re
import time
import os

In [ ]:
# 엑셀에 여러 시트로 저장하기 위한 모듈
!pip install Xlsxwriter

# 데이터수집
부문별로 나누어 저장(타자, 투수, 수비, 주루)

In [ ]:
driver = webdriver.Chrome("C:/Users/wbs04/workspace/chromedriver_win32/chromedriver.exe")

In [ ]:
def go_to_page(selector):
    global soup
    click_on = driver.find_element(By.CSS_SELECTOR, selector)
    click_on.click()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
def make_dataframe():
    col_names = []
    record_col = soup.select("#cphContents_cphContents_cphContents_udpContent > div.record_result > table > thead > tr > th")
    for item in record_col:
        col_names.append(item.text)
        
    return pd.DataFrame(columns=col_names)

In [ ]:
# 해당 url로 이동
url = "https://www.koreabaseball.com"
driver.get(url)

In [ ]:
# 기록 -> 기록실로 이동
go_to_page('#lnb > li:nth-child(3) > a')
go_to_page('#lnb02')

## 타자기록
2002년 이후 기록 값들이 바뀜, 세부기록 추가됨

### 1982~2001 타자기록

In [ ]:
# 타자부분에서 연도 선택 -> 팀 선택 -> 페이지 선택
# hitter_dict = {{연도1:{팀1:기록, 팀2:기록, ...}}, {연도2:...}, ...}
# 연도를 key값으로 갖고, value를 '팀을 key, 기록 데이터프레임을 value로 하는 딕셔너리'로 하는 딕셔너리

# hitter_record 폴더 -> 연도별 엑셀 파일 -> 팀별 시트

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
hitter_dict = {}
for idx, year in enumerate(years[:-22]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = pd.DataFrame(columns=col_names)

        team_name = team.text.strip()
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    hitter_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in hitter_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'hitter_record/{k}.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        hitter_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

### 2002~2022 타자기록

#### 기본기록

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
hitter_dict = {}
for idx, year in enumerate(years[-22:-1]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+21})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = pd.DataFrame(columns=col_names)

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    hitter_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in hitter_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'hitter_record/{k}.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        hitter_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

#### 세부기록

In [ ]:
# 세부기록으로 이동
go_to_page('#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a')

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
hitter_dict = {}
for idx, year in enumerate(years[-21:]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+21})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = pd.DataFrame(columns=col_names)

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    hitter_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in hitter_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'hitter_record/{k}_details.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        hitter_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

## 투수기록
스크래핑 방법 변경
- 2002년 기준으로 나누어 스크래핑 -> 연도별 데이터프레임 새로 만든 후 전체 스크래핑(make_dataframe 함수 설계)
- 전체 스크래핑 후 2002년 이후의 세부기록 추가

In [ ]:
# 투수로 이동
go_to_page('#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(2)')

### 1982~2022 투수기록

In [ ]:
df_record = make_dataframe()

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
pitcher_dict = {}
for idx, year in enumerate(years[:-1]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = make_dataframe()

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    pitcher_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in pitcher_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'pitcher_record/{k}.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        pitcher_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

### 2002~2022 투수 세부기록

In [ ]:
# 세부기록으로 이동
go_to_page('#cphContents_cphContents_cphContents_udpContent > div.row > div.tab-depth3 > ul > li:nth-child(2) > a')

In [ ]:
df_record = make_dataframe()

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
pitcher_dict = {}
for idx, year in enumerate(years[-21:]):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+21})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = make_dataframe()

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    pitcher_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in pitcher_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'pitcher_record/{k}_details.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        pitcher_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

## 수비기록
2001년 이전 자료 없음

In [ ]:
# 투수로 이동
go_to_page('#contents > div.sub-content > div.tab-depth2 > ul > li:nth-child(3)')

### 2001~2022 수비기록

In [ ]:
df_record = make_dataframe()

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
defence_dict = {}
for idx, year in enumerate(years):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = make_dataframe()

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    defence_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in defence_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'defence_record/{k}.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        defence_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

## 주루기록
2001년 이전 자료 없음

### 2001~2022 주루기록

In [ ]:
df_record = make_dataframe()

In [ ]:
years = soup.select('#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option')
run_dict = {}
for idx, year in enumerate(years):
    season = year.text
    print(f"{season}시즌")  # 스크래핑 완료 확인용
    go_to_page(f'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason > option:nth-child({idx+1})')


    # 전체 팀에 대해 자료 가져오기(년도별)
    team_dict = {}
    teams = soup.select('#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option')
    for idx, team in enumerate(teams[1:]):
        df_record = make_dataframe()

        team_name = team.text
        print(idx+1, team_name)  # 스크래핑 완료 확인용
        go_to_page(f'#cphContents_cphContents_cphContents_ddlTeam_ddlTeam > option:nth-child({idx+2})')

        # 페이지 길이
        # 페이지 번호 길이
        page_btn = soup.select("div.paging > a")     
        page_range = 2 if len(page_btn) == 1 else len(page_btn)-1

        for j in range(1, page_range):
            # 페이지 이동
            go_to_page(f'#cphContents_cphContents_cphContents_ucPager_btnNo{j}')

            # 데이터프레임에 선수 정보 한 행씩 추가
            players_list = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')
            for player in players_list:
                player_record = player.text.strip().split('\n')
                df_record.loc[len(df_record)] = player_record

        team_dict[team_name] = df_record
    run_dict[season] = team_dict
    print('='*10)

In [ ]:
# 시트에 각각 저장
for k, v in run_dict.items():
    # 년도별 파일 생성
    writer = pd.ExcelWriter(f'run_record/{k}.xlsx', engine='xlsxwriter')
    for item in v.keys():
        # 시트에 팀별로 저장
        run_dict[k][item].to_excel(writer, sheet_name=item, index=False)
    # 파일 저장
    writer.save()

## 데이터정리
부문별 구별되어 있는 데이터를 연도별로 나누어 팀별로 구분

In [ ]:
# 폴더 생성
os.mkdir('by_year')
for i in range(1982, 2023):
    os.mkdir(f'by_year/{i}')

In [ ]:
record_files = ['hitter_record', 'pitcher_record', 'defence_record', 'run_record']

In [ ]:
for file in record_files:
    s = ' '.join(os.listdir(f'{file}'))
    pat = re.compile('(\d{4})[.]xlsx')
    years = pat.findall(s)
    for year in years:
        df_record = pd.ExcelFile(f'{file}/{year}.xlsx')
        record_sheets = df_record.sheet_names
        
        for item in record_sheets:
            if os.path.exists(f'by_year/{year}/{item.strip()}.xlsx'):
                writer = pd.ExcelWriter(f'by_year/{year}/{item.strip()}.xlsx', mode='a', engine='openpyxl')
            else:
                writer = pd.ExcelWriter(f'by_year/{year}/{item.strip()}.xlsx', engine='xlsxwriter')
                
            df_record.parse(item).to_excel(writer, sheet_name=file, index=False)
            writer.save()
            
writer.close()

In [2]:
# 세부기록 추가
# '연도'_details.xlsx 파일로 저장

In [ ]:
for file in record_files[:2]:
    s = ' '.join(os.listdir(f'{file}'))
    pat = re.compile('(\d{4})_details[.]xlsx')
    years = pat.findall(s)
    for year in years:
        df_record = pd.ExcelFile(f'{file}/{year}.xlsx')
        record_sheets = df_record.sheet_names
        
        for item in record_sheets:
            writer = pd.ExcelWriter(f'by_year/{year}/{item.strip()}.xlsx', mode='a', engine='openpyxl')
            df_record.parse(item).to_excel(writer, sheet_name=f'{file}_details', index=False)
            writer.save()
            
writer.close()

## 연도별 구단확인

In [ ]:
# 팀 리스트 출력 함수
# 연도별로 저장한 
def team_list(season):
    team_name = []
    for item in os.listdir(f'by_year/{season}'):
        team_name.append(re.sub('(.+?)[.]xlsx', '\g<1>', item))
        
    return team_name

In [ ]:
# 전년도와 비교해서 팀구성이 달라진 연도 저장
change_list = []
for i in range(1983, 2023):
    if team_list(i) != show_team_list(i-1):
        change_list.append(i)

In [ ]:
# 확인
print(f'1982 ~ : {team_list(1982)}')
for season in change_list:
    print(f'{season} ~ : {team_list(season)}')

### 연도별 순위확인 - 포스트시즌

In [ ]:
# 구단 변천사 확인
# https://www.koreabaseball.com/History/Etc/TeamHistory.aspx
# https://namu.wiki/w/KBO%20%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C 포스트시즌
# https://ko.wikipedia.org/wiki/KBO_%EB%A6%AC%EA%B7%B8_%EC%97%B0%EB%8F%84%EB%B3%84_%ED%8C%80_%EC%88%9C%EC%9C%84

In [ ]:
import urllib.request
from urllib.request import Request, urlopen  # HTTPError: HTTP Error 403: Forbidden 에러발생

In [ ]:
# 포스트시즌 기록
url = 'https://namu.wiki/w/KBO%20%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
data = urlopen(req).read()
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
table = soup.select('#jWZtJ8Cjb > div.aa7ee88d > div > div > div > div > div > div > div > div > div > div.aCNBxUDm > div:nth-child(7) > div > div > div > div > div > div > div > div > div:nth-child(6) > div > div:nth-child(13) > div > div > div > div > div > div > div > div:nth-child(1) > div > div:nth-child(21) > div.rYXEqDWZ > table > tbody')[0]

In [ ]:
cols = []
for i in range(6):
    cols.append(table.select('.upwzFoAB')[i].text)
post_season = pd.DataFrame(columns=cols)

In [ ]:
for i in range(1, 41):
    elements = []
    for j in range(i*6, (i+1)*6):
        elements.append(table.select('.upwzFoAB')[j].text)
    post_season.loc[i-1] = elements

In [ ]:
# 연도 인덱스 설정 후 엑셀 파일로 저장
post_season = post_season.set_index('연도')
post_season.to_excel('post_season.xlsx')

### 연도별 순위확인 - 정규시즌

In [ ]:
url = 'https://www.koreabaseball.com/TeamRank/TeamRank.aspx'
data = urllib.request.urlopen(url)
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
url = "https://www.koreabaseball.com/TeamRank/TeamRank.aspx"
driver.get(url)

In [ ]:
cols = ['연도', '1위', '2위', '3위', '4위', '5위', '6위', '7위', '8위', '9위', '10위']
regular_season = pd.DataFrame(columns=cols)

In [ ]:
for i in reversed(range(1, 42)):
    go_to_page(f'#cphContents_cphContents_cphContents_ddlYear > option:nth-child({i})')
    
    rank_list = soup.select('#cphContents_cphContents_cphContents_udpRecord > table > tbody > tr')
    ranking = [soup.select(f'#cphContents_cphContents_cphContents_ddlYear > option:nth-child({i})')[0].text]
    for rank in rank_list:
        ranking.append(rank.select('td')[1].text)
    while len(ranking) != 11:
        ranking.append('-')
    regular_season.loc[i-1] = ranking

In [ ]:
regular_season = regular_season.set_index('연도')

In [ ]:
regular_season.loc['2000'] = ['현대', '두산', '삼성', 'LG', '롯데', '해태', '한화', 'SK', '-', '-']
regular_season.loc['1999'] = ['두산', '롯데', '삼성', '한화', '현대', 'LG', '해태', '쌍방울', '-', '-']

In [ ]:
regular_season.to_excel('regular_season.xlsx')

### 포스트시즌 전 순위

In [ ]:
before_post_season = regular_season

In [ ]:
indexs = post_season.index
pat = re.compile('\((\d위)\)')

In [ ]:
for i in indexs[19:]:
    for v in post_season.loc[i].values:
        if len(v):
            before_post_season[pat.findall(v)[0]][i] = v.split()[0].upper()

In [ ]:
before_post_season['6위']['2009'] = '넥센'

In [ ]:
before_post_season.to_excel('before_post_season.xlsx')

### 팀별순위

In [ ]:
url = "https://www.koreabaseball.com/History/Etc/TeamHistory.aspx"
driver.get(url)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
rows = soup.select('#change > table > tbody > tr')

In [ ]:
team_ranking_list = []

In [ ]:
for i in range(12):
    tmp = []
    for row in rows:
        tmp.append(row.select('td')[i].text.strip())
    team_ranking_list.append(tmp)

In [ ]:
team_ranking_list

In [ ]:
pat = re.compile('(.+?)\\n창단일')
name_change = []
for i in range(len(team_ranking_list)):
    tmp = []
    for item in team_ranking_list[i]:
        if pat.findall(item):
            tmp.append(pat.findall(item)[0])
    name_change.append(tmp)

In [ ]:
name_change

In [ ]:
pat = re.compile('(\d+)\D')
for i in range(12):
    for j in range(40):
        rank = pat.findall(team_ranking_list[i][j])
        if rank:
            team_ranking_list[i][j] = pat.findall(team_ranking_list[i][j])[0]
            if len(team_ranking_list[i][j]) > 2:
                team_ranking_list[i][j] = ''

In [ ]:
team_ranking_list

In [ ]:
years = [x.select('th')[0].text for x in rows]

In [ ]:
df = pd.DataFrame(team_ranking_list).T
df.columns = [x[-1] for x in name_change]
df.index = [x for x in range(1982, 2022)]
df.index.name = '연도'

In [ ]:
df.columns = ['삼성', 'KIA', '롯데', '현대', 'LG', '두산', '한화', '쌍방울', 'SSG', '키움', 'NC', 'KT']
df

In [ ]:
df.apply(pd.to_numeric)
df

In [ ]:
df.to_excel('team_ranking.xlsx')